In [1]:
# When using Colab, check the GPU that is assigned and reload the runtime if its memory is low
!nvidia-smi

Thu May 27 10:02:51 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
!ls "/content/drive/My Drive/GSoC2021/datasets/PilotNet"
!unzip "/content/drive/My Drive/GSoC2021/datasets/PilotNet/complete_dataset.zip"
!unzip "/content/drive/My Drive/GSoC2021/datasets/PilotNet/curves_only.zip"

Streaming output truncated to the last 5000 lines.
  inflating: curves_only/Images/5059.png  
  inflating: curves_only/Images/103.png  
  inflating: curves_only/Images/5253.png  
  inflating: curves_only/Images/1293.png  
  inflating: curves_only/Images/3470.png  
  inflating: curves_only/Images/2916.png  
  inflating: curves_only/Images/394.png  
  inflating: curves_only/Images/3013.png  
  inflating: curves_only/Images/365.png  
  inflating: curves_only/Images/2831.png  
  inflating: curves_only/Images/844.png  
  inflating: curves_only/Images/3602.png  
  inflating: curves_only/Images/3067.png  
  inflating: curves_only/Images/4491.png  
  inflating: curves_only/Images/5165.png  
  inflating: curves_only/Images/1542.png  
  inflating: curves_only/Images/51.png  
  inflating: curves_only/Images/3188.png  
  inflating: curves_only/Images/732.png  
  inflating: curves_only/Images/4628.png  
  inflating: curves_only/Images/3391.png  
  inflating: curves_only/Images/2532.png  
  inflatin

In [10]:
import os
import glob
import numpy as np
import cv2
from tqdm import tqdm


def load_data(folder):
    name_folder = folder + '/Images/'
    list_images = glob.glob(name_folder + '*')
    images = sorted(list_images, key=lambda x: int(x.split('/')[-1].split('.png')[0]))
    name_file = folder + '/data.json'
    file = open(name_file, 'r')
    data = file.read()
    file.close()
    return images, data

def get_images(list_images, type_image, array_imgs):
    # Read the images
    for name in tqdm(list_images):
        img = cv2.imread(name)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        if type_image == 'cropped':
            img = img[240:480, 0:640]
        # img = cv2.resize(img, (int(img.shape[1] / 4), int(img.shape[0] / 4)))
        img = cv2.resize(img, (int(200), int(66)))
        array_imgs.append(img)

    return array_imgs

def parse_json(data, array):
    # Process json
    data_parse = data.split('}')[:-1]
    for d in data_parse:
        v = d.split('"v": ')[1]
        d_parse = d.split(', "v":')[0]
        w = d_parse.split(('"w": '))[1]
        array.append((float(v), float(w)))

    return array

def preprocess_data(array, imgs):
    # Data augmentation
    # Take the image and just flip it and negate the measurement
    flip_imgs = []
    array_flip = []
    for i in tqdm(range(len(imgs))):
        flip_imgs.append(cv2.flip(imgs[i], 1))
        array_flip.append((array[i][0], -array[i][1]))
    new_array = array + array_flip
    new_array_imgs = imgs + flip_imgs
    return new_array, new_array_imgs

def normalize(x):
    x = np.asarray(x)
    return (x - x.min()) / (np.ptp(x))

def check_path(path):
    if not os.path.exists(path):
        print(f"{path} not exist")
        os.makedirs(path)
        print(f"Create {path} success")

In [11]:
from torch.utils.data.dataset import Dataset
from torchvision import transforms
from PIL import Image

class PilotNetDataset(Dataset):
    def __init__(self, path_to_data, path_to_data_curve, transforms=None):

        self.data_path = path_to_data

        all_images, all_data = load_data(path_to_data)
        all_images_curve, all_data_curve = load_data(path_to_data_curve)

        type_image = 'cropped'

        self.images = []
        self.images = get_images(all_images, type_image, self.images)
        # self.images = get_images(all_images_curve, type_image, self.images)

        self.labels = []
        self.labels = parse_json(all_data, self.labels)
        # self.labels = parse_json(all_data_curve, self.labels)

        self.labels, self.images = preprocess_data(self.labels, self.images)

        self.transforms = transforms

        self.image_shape = self.images[0].shape
        self.num_labels = np.array(self.labels[0]).shape[0]

        self.count = len(self.images)
        
    def __getitem__(self, index):

        img = self.images[index]
        label = np.array(self.labels[index])
        data = Image.fromarray(img)

        if self.transforms is not None:
            data = self.transforms(data)

        return (data, label)

    def __len__(self):
        return self.count

In [12]:
import torch
import torch.nn as nn


class PilotNet(nn.Module):
    def __init__(self,
                image_shape,
                num_labels):
        super(PilotNet, self).__init__()

        self.img_height = image_shape[0]
        self.img_width = image_shape[1]
        self.num_channels = image_shape[2]

        self.output_size = num_labels
        
        self.ln_1 = nn.BatchNorm2d(self.num_channels, eps=1e-03)

        self.cn_1 = nn.Conv2d(self.num_channels, 24, kernel_size=5, stride=2)
        self.cn_2 = nn.Conv2d(24, 36, kernel_size=5, stride=2)
        self.cn_3 = nn.Conv2d(36, 48, kernel_size=5, stride=2)
        self.cn_4 = nn.Conv2d(48, 64, kernel_size=3, stride=1)
        self.cn_5 = nn.Conv2d(64, 64, kernel_size=3, stride=1)

        self.fc_1 = nn.Linear(1 * 18 * 64, 1164)
        self.fc_2 = nn.Linear(1164, 100)
        self.fc_3 = nn.Linear(100, 50)
        self.fc_4 = nn.Linear(50, 10)
        self.fc_5 = nn.Linear(10, self.output_size)

    def forward(self, img):

        out = self.ln_1(img)

        out = self.cn_1(out)
        out = torch.relu(out)
        out = self.cn_2(out)
        out = torch.relu(out)
        out = self.cn_3(out)
        out = torch.relu(out)
        out = self.cn_4(out)
        out = torch.relu(out)
        out = self.cn_5(out)
        out = torch.relu(out)

        out = out.reshape(out.size(0), -1)

        out = self.fc_1(out)
        out = torch.relu(out)
        out = self.fc_2(out)
        out = torch.relu(out)
        out = self.fc_3(out)
        out = torch.relu(out)
        out = self.fc_4(out)
        out = torch.relu(out)
        out = self.fc_5(out)

        return out

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch.utils.tensorboard import SummaryWriter

import argparse
from PIL import Image

import json
import numpy as np

# Base Directory
path_to_data = './complete_dataset'
path_to_data_curves = './curves_only'
base_dir = './experiments/'+ 'exp_random' + '/'
model_save_dir = base_dir + 'trained_models'
log_dir = base_dir + 'log'

check_path(base_dir)
check_path(log_dir)
check_path(model_save_dir)

# Hyperparameters
num_epochs = 50
batch_size = 256
learning_rate = 1e-3
test_split = 0.2
shuffle_dataset = True
save_iter = 50
random_seed = 42
print_terminal = True

# Device Selection (CPU/GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
FLOAT = torch.FloatTensor

# Tensorboard Initialization
writer = SummaryWriter(log_dir)

# Define data transformations
transformations = transforms.Compose([
                                transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
                                transforms.GaussianBlur(5, sigma=(0.1, 2.0)),
                                transforms.ToTensor()
                            ])
# Load data
dataset = PilotNetDataset(path_to_data, path_to_data_curves, transformations)

# Creating data indices for training and test splits:
dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(test_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, test_split = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
test_sampler = SubsetRandomSampler(test_split)

train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
test_loader = DataLoader(dataset, batch_size=batch_size, sampler=test_sampler)

# Load Model
pilotModel = PilotNet(dataset.image_shape, dataset.num_labels).to(device)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(pilotModel.parameters(), lr=learning_rate)

# Train the model
total_step = len(train_loader)
loss_list = []
acc_list = []
global_iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        
        images = FLOAT(images).to(device)
        labels = FLOAT(labels.float()).to(device)
        
        # Run the forward pass
        outputs = pilotModel(images)
        loss = criterion(outputs, labels)
        current_loss = loss.item()

        # Backprop and perform Adam optimisation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Track the accuracy
        total = labels.size(0)
        correct = (torch.linalg.norm(outputs - labels) < 0.1).sum().item()
        current_acc = (correct / total)

        if global_iter % save_iter == 0:
            torch.save(pilotModel.state_dict(), model_save_dir + '/pilot_net_model_{}.ckpt'.format(random_seed))

        global_iter += 1

        writer.add_scalar("performance/loss", current_loss, global_iter)
        writer.add_scalar("performance/accuracy", current_acc, global_iter)
        writer.add_scalar("training/epochs", epoch+1, global_iter)

        if print_terminal and (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                        (correct / total) * 100))

# Test the model
pilotModel.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = FLOAT(images).to(device)
        labels = FLOAT(labels.float()).to(device)
        outputs = pilotModel(images)
        total += labels.size(0)
        correct += (torch.linalg.norm(outputs - labels) < 0.1).sum().item()

    print('Test Accuracy of the model on the test images: {} %'.format((correct / total) * 100))

# Save the model and plot
torch.save(pilotModel.state_dict(), model_save_dir + '/pilot_net_model_{}.ckpt'.format(random_seed))

./experiments/exp_random/ not exist
Create ./experiments/exp_random/ success
./experiments/exp_random/log not exist
Create ./experiments/exp_random/log success
./experiments/exp_random/trained_models not exist
Create ./experiments/exp_random/trained_models success


100%|██████████| 17341/17341 [00:01<00:00, 17262.65it/s]


Epoch [1/50], Step [100/109], Loss: 3.4495, Accuracy: 0.00%
Epoch [2/50], Step [100/109], Loss: 2.6437, Accuracy: 0.00%
Epoch [3/50], Step [100/109], Loss: 2.0263, Accuracy: 0.00%
Epoch [4/50], Step [100/109], Loss: 2.0901, Accuracy: 0.00%
Epoch [5/50], Step [100/109], Loss: 1.9630, Accuracy: 0.00%
Epoch [6/50], Step [100/109], Loss: 1.4013, Accuracy: 0.00%
Epoch [7/50], Step [100/109], Loss: 1.3743, Accuracy: 0.00%
Epoch [8/50], Step [100/109], Loss: 1.4285, Accuracy: 0.00%
Epoch [9/50], Step [100/109], Loss: 1.0241, Accuracy: 0.00%
Epoch [10/50], Step [100/109], Loss: 1.3716, Accuracy: 0.00%
Epoch [11/50], Step [100/109], Loss: 1.1968, Accuracy: 0.00%
Epoch [12/50], Step [100/109], Loss: 1.2773, Accuracy: 0.00%
Epoch [13/50], Step [100/109], Loss: 1.1078, Accuracy: 0.00%
Epoch [14/50], Step [100/109], Loss: 1.1002, Accuracy: 0.00%
Epoch [15/50], Step [100/109], Loss: 0.9323, Accuracy: 0.00%
Epoch [16/50], Step [100/109], Loss: 1.0025, Accuracy: 0.00%
Epoch [17/50], Step [100/109], Lo

In [15]:
# When using Colab, check the GPU that is assigned and reload the runtime if its memory is low
!nvidia-smi

Thu May 27 12:06:16 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    66W / 149W |    882MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [16]:
num_epochs = 100

# Train the model
total_step = len(train_loader)
loss_list = []
acc_list = []
global_iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        
        images = FLOAT(images).to(device)
        labels = FLOAT(labels.float()).to(device)
        
        # Run the forward pass
        outputs = pilotModel(images)
        loss = criterion(outputs, labels)
        current_loss = loss.item()

        # Backprop and perform Adam optimisation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Track the accuracy
        total = labels.size(0)
        correct = (torch.linalg.norm(outputs - labels) < 0.1).sum().item()
        current_acc = (correct / total)

        if global_iter % save_iter == 0:
            torch.save(pilotModel.state_dict(), model_save_dir + '/pilot_net_model_{}.ckpt'.format(random_seed))

        global_iter += 1

        writer.add_scalar("performance/loss", current_loss, global_iter)
        writer.add_scalar("performance/accuracy", current_acc, global_iter)
        writer.add_scalar("training/epochs", epoch+1, global_iter)

        if print_terminal and (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                        (correct / total) * 100))

# Test the model
pilotModel.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = FLOAT(images).to(device)
        labels = FLOAT(labels.float()).to(device)
        outputs = pilotModel(images)
        total += labels.size(0)
        correct += (torch.linalg.norm(outputs - labels) < 0.1).sum().item()

    print('Test Accuracy of the model on the test images: {} %'.format((correct / total) * 100))

# Save the model and plot
torch.save(pilotModel.state_dict(), model_save_dir + '/pilot_net_model_{}.ckpt'.format(random_seed))

Epoch [1/100], Step [100/109], Loss: 0.2239, Accuracy: 0.00%
Epoch [2/100], Step [100/109], Loss: 0.2964, Accuracy: 0.00%
Epoch [3/100], Step [100/109], Loss: 0.3971, Accuracy: 0.00%
Epoch [4/100], Step [100/109], Loss: 0.3453, Accuracy: 0.00%
Epoch [5/100], Step [100/109], Loss: 0.1605, Accuracy: 0.00%
Epoch [6/100], Step [100/109], Loss: 0.1019, Accuracy: 0.00%
Epoch [7/100], Step [100/109], Loss: 0.4687, Accuracy: 0.00%
Epoch [8/100], Step [100/109], Loss: 0.1878, Accuracy: 0.00%
Epoch [9/100], Step [100/109], Loss: 0.1981, Accuracy: 0.00%
Epoch [10/100], Step [100/109], Loss: 0.2133, Accuracy: 0.00%
Epoch [11/100], Step [100/109], Loss: 0.1904, Accuracy: 0.00%
Epoch [12/100], Step [100/109], Loss: 0.3654, Accuracy: 0.00%
Epoch [13/100], Step [100/109], Loss: 0.1414, Accuracy: 0.00%
Epoch [14/100], Step [100/109], Loss: 0.1263, Accuracy: 0.00%
Epoch [15/100], Step [100/109], Loss: 0.1648, Accuracy: 0.00%
Epoch [16/100], Step [100/109], Loss: 0.2786, Accuracy: 0.00%
Epoch [17/100], S

In [23]:
pilotModel.eval()
with torch.no_grad():
    correct = 0
    total = 0
    losses = []
    for images, labels in test_loader:
        images = FLOAT(images).to(device)
        labels = FLOAT(labels.float()).to(device)
        outputs = pilotModel(images)

        losses.append(nn.MSELoss()(outputs[1], labels[1]).item())

print(np.mean(losses), np.max(losses))

0.014512438299009642 0.05942890793085098
